# Multi-agent E-Commerce Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew

In [5]:
import os

openai_api_key = "sk-proj-"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [6]:
customer_agent = Agent(
    role="E-commerce Customer Agent",
    goal="Help customers find the perfect products and ensure a smooth buying experience",
    backstory=(
        "You are a virtual customer agent for Amazon. "
        "Your goal is to understand customer needs, recommend relevant products, guide them through the purchase process, and address any questions or concerns they may have. "
        "Today, you're assisting {customer_name}, who's looking for women clothes. "
        "Be helpful, informative, and make their shopping experience a breeze!"
    ),
    allow_delegation=False,
    verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [7]:
customer_qa_agent = Agent(
    role="E-commerce Customer QA Agent",
    goal="Ensure customer agents provide exceptional support "
         "and accurate information",
    backstory=(
        "You are a Customer QA Agent for Amazon. "
        "Your job is to analyze customer interactions and ensure agents are delivering "
        "high-quality support. Today, you're reviewing a conversation between "
        "customer {customer_name}  and agent {agent_name} . "
        "Make sure the agent provided accurate product information, addressed "
        "customer concerns effectively, and followed company guidelines."
    ),
    verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [9]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [10]:
web_scrape_tool = WebsiteSearchTool(
    website_url="https://www.amazon.com.tr/s?i=fashion&bbn=13546651031&rh=n%3A12466553031%2Cn%3A13546651031%2Cp_n_deal_type%3A26902947031&_encoding=UTF8&content-id=amzn1.sym.f71b2bd5-89a0-43f5-bbce-a52739964468&pd_rd_r=ca0410b3-513f-40eb-b5cc-b05db6762bab&pd_rd_w=276Ho&pd_rd_wg=mE1ul&pf_rd_p=f71b2bd5-89a0-43f5-bbce-a52739964468&pf_rd_r=YDGPX2NG14YR7MCDJRCW&ref=pd_hp_d_atf_unk"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [11]:
customer_agent_task = Task(
    description=(
        "A customer named {customer_name} has reached out with the following inquiry: "
        "{inquiry}\n\n"
        "Your goal is to provide a clear, concise, and helpful response to the customer. "
        "Use your knowledge base and available tools to find the best solution. "
        "If you cannot find a solution, escalate the issue to a higher-level agent. "
        "Remember to maintain a friendly and professional tone throughout the interaction."
    ),
    expected_output=(
        "A comprehensive and informative response to the customer's inquiry. "
        "The response should be easy to understand and address all aspects of the customer's question. "
        "If applicable, provide next steps or solutions. "
        "Ensure the customer feels satisfied with the resolution."
    ),
	tools=[web_scrape_tool],
    agent=customer_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [12]:
customer_qa_task = Task(
    description=(
        "Review the customer interaction between agent {agent_name} and customer {customer_name}. "
        "Assess the agent's response for accuracy, completeness, and adherence to company standards. "
        "Evaluate the agent's ability to understand and address the customer's needs, "
        "maintain a professional and friendly tone, and resolve the issue effectively. "
        "Identify areas for improvement and provide constructive feedback."
    ),
    expected_output=(
        "A comprehensive evaluation of the customer interaction, including strengths and weaknesses. "
        "Provide specific feedback on the agent's performance, such as areas for improvement "
        "and suggestions for enhancing future interactions. "
        "Identify any potential training needs or process improvements."
    ),
    agent=customer_qa_agent
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [13]:
crew = Crew(
  agents=[customer_agent, customer_qa_agent],
  tasks=[customer_agent_task, customer_qa_task],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [15]:
inputs = {
    "customer_name": "Ayşe",
    "agent_name" : customer_agent,
    "inquiry": "I need help find a white dress.The dress must be short."
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: E-commerce Customer Agent
 [INFO]: == Starting Task: A customer named Ayşe has reached out with the following inquiry: I need help find a white dress.The dress must be short.

Your goal is to provide a clear, concise, and helpful response to the customer. Use your knowledge base and available tools to find the best solution. If you cannot find a solution, escalate the issue to a higher-level agent. Remember to maintain a friendly and professional tone throughout the interaction.


> Entering new CrewAgentExecutor chain...
I need to assist Ayşe in finding a white short dress. I should use the "Search in a specific website" tool to search for white short dresses on Amazon's website.

Action: Search in a specific website
Action Input: {"search_query": "white short dress", "website": "Amazon"} 

Relevant Content:


Title: Women's Short Sleeve V Neck Casual Swing Dress

Description: This white short dress is made of high-quality fabric and features a flattering v

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Upon evaluating the customer interaction between agent Ayşe and customer Ayşe, I found that the agent responded accurately and effectively to the customer's inquiry for a white short dress. The agent utilized the "Search in a specific website" tool to search for white short dresses on Amazon's website, which aligns with company standards and demonstrates a proactive approach to addressing customer needs.

Strengths:
1. Accuracy: The agent accurately identified the customer's request for a white short dress.
2. Efficient use of tools: The agent efficiently utilized the search tool to find relevant products.
3. Professional and friendly tone: The agent maintained a professional and friendly tone throughout the interaction.

Areas for Improvement:
1. Product recommendation: While the agent found relevant white dresses, they could have provided more options to the customer for a wider selection.
2. Product details: The agent could have provided more detailed information about the recommended products to assist the customer in making an informed decision.

Suggestions for Enhancing Future Interactions:
1. Offer a variety of options: Provide the customer with a range of white short dresses to choose from, catering to different styles and preferences.
2. Provide detailed product information: Include specific details such as material, sizing, and customer reviews to help the customer make a confident purchase decision.

Potential Training Needs or Process Improvements:
1. Training on product knowledge: Ensure agents are well-versed in the details of various products to offer comprehensive assistance to customers.
2. Continuous feedback: Implement a feedback system to gather insights from customers and agents for ongoing improvement in customer interactions.

Overall, the agent demonstrated a good understanding of the customer's needs and maintained a positive interaction. By enhancing product recommendations and providing more detailed information, future interactions can be further improved to create a seamless shopping experience for customers.

In [23]:
from crewai_tools import FileReadTool
tecnology_info = FileReadTool(
    file_path="info.txt"
)

In [24]:
hr_agent = Agent(
    role="Senior HR Manager",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at software company"
        "and creating a job posting  "
        "seeking for {job_title} for your company."
        "You need to make sure that you provide the best job post and desciption!"
    ),
    tools=[tecnology_info],
    allow_delegation=False,
    verbose=True
)

In [35]:
# Create a task
task1 = Task(
    description="Create a job title using information for {company}.",
    expected_output="A job posting with a detailed job description, responsibilities, using technologies,nice to have and benefits provided by the company.",
    output_file="task1output.txt",
    agent=hr_agent,
)


In [36]:
crew_hr = Crew(agents=[hr_agent], tasks=[task1], verbose=1)


2024-08-17 01:34:56,752 - 139753787572352 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [37]:
inputs = {
    "job_title": "Python Backend Developer",
    "company": "HelloART",}
result = crew_hr.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior HR Manager


> Entering new CrewAgentExecutor chain...
I need to create a job posting for a Python Backend Developer position at our software company. I should include a detailed job description, responsibilities, required technologies, nice to have qualifications, and benefits provided by the company.

Action: Read a file's content
Action Input: {"file": "info.txt"} 

PostgreSQL,NoSQL,englsih,AWS etc,Msc,Bachelor,Agile

CI/CD,

Job Title: Python Backend Developer at HelloART

Job Description:
HelloART is seeking a talented Python Backend Developer to join our dynamic team. As a Python Backend Developer, you will be responsible for developing and maintaining the backend of our software applications. You will work closely with our frontend developers, product managers, and other team members to deliver high-quality, scalable solutions.

Responsibilities:
- Develop and maintain backend services using Python
- Design and implement database schemas using 

In [38]:
from IPython.display import Markdown
Markdown(result)

Job Title: Python Backend Developer at HelloART

Job Description:
HelloART is seeking a talented Python Backend Developer to join our dynamic team. As a Python Backend Developer, you will be responsible for developing and maintaining the backend of our software applications. You will work closely with our frontend developers, product managers, and other team members to deliver high-quality, scalable solutions.

Responsibilities:
- Develop and maintain backend services using Python
- Design and implement database schemas using PostgreSQL and NoSQL databases
- Collaborate with frontend developers to integrate user-facing elements with server-side logic
- Optimize applications for maximum speed and scalability
- Implement security and data protection measures
- Participate in code reviews and provide constructive feedback to other team members
- Stay up-to-date on emerging technologies and best practices in backend development

Required Technologies:
- Proficiency in Python
- Experience working with PostgreSQL and NoSQL databases
- Familiarity with AWS services
- Strong understanding of software development principles
- Bachelor's or Master's degree in Computer Science or a related field
- Knowledge of Agile methodologies

Nice to Have:
- Experience with CI/CD pipelines
- Knowledge of containerization technologies such as Docker
- Familiarity with frontend technologies such as React or Angular

Benefits:
- Competitive salary
- Health, dental, and vision insurance
- Flexible work hours
- Remote work options
- Professional development opportunities
- Fun and collaborative work environment

If you are passionate about backend development and want to work with a talented team on cutting-edge projects, apply now to join HelloART as a Python Backend Developer!